In [ ]:
%pip install google-analytics-data

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    Dimension,
    DateRange,
    MinuteRange,
    Metric,
    MetricType,
    RunReportRequest,
    RunRealtimeReportRequest,
)

In [ ]:
def print_run_report_response(response):
    """Prints results of a runReport call."""
    # [START analyticsdata_print_run_report_response_header]
    print(f"{response.row_count} rows received")
    for dimensionHeader in response.dimension_headers:
        print(f"Dimension header name: {dimensionHeader.name}")
    for metricHeader in response.metric_headers:
        metric_type = MetricType(metricHeader.type_).name
        print(f"Metric header name: {metricHeader.name} ({metric_type})")
    # [END analyticsdata_print_run_report_response_header]

    # [START analyticsdata_print_run_report_response_rows]
    print("Report result:")
    for rowIdx, row in enumerate(response.rows):
        print(f"\nRow {rowIdx}")
        for i, dimension_value in enumerate(row.dimension_values):
            dimension_name = response.dimension_headers[i].name
            print(f"{dimension_name}: {dimension_value.value}")

        for i, metric_value in enumerate(row.metric_values):
            metric_name = response.metric_headers[i].name
            print(f"{metric_name}: {metric_value.value}")

In [ ]:
def sample_run_report(property_id=os.getenv("GA4_PROPERTY_ID")):
    """Runs a simple report on a Google Analytics 4 property."""
    # TODO(developer): Uncomment this variable and replace with your
    #  Google Analytics 4 property ID before running the sample.
    # property_id = "YOUR-GA4-PROPERTY-ID"

    # Using a default constructor instructs the client to use the credentials
    # specified in GOOGLE_APPLICATION_CREDENTIALS environment variable.
    client = BetaAnalyticsDataClient()

    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[Dimension(name="hour")],
        metrics=[Metric(name="newUsers"), Metric(name="totalUsers")],
        date_ranges=[DateRange(start_date="2024-03-21", end_date="2024-03-22")],
    )
    response = client.run_report(request)

    resp = []

    for row in response.rows:
        resp.append(
            {
                "hour": int(row.dimension_values[0].value),
                "new_users": int(row.metric_values[0].value),
                "total_users": int(row.metric_values[1].value),
            }
        )

    resp = sorted(resp, key=lambda x: x["hour"])

    return resp

In [ ]:
sample = sample_run_report(os.getenv("GA4_PROPERTY_ID"))

In [ ]:
def run_realtime_report(property_id):
    """Runs a realtime report on a Google Analytics 4 property."""
    client = BetaAnalyticsDataClient()

    request = RunRealtimeReportRequest(
        property=f"properties/{property_id}",
        dimensions=[Dimension(name="country")],
        metrics=[Metric(name="activeUsers")],
    )
    response = client.run_realtime_report(request)

    count = 0

    for row in response.rows:
        count += row.metric_values[0].value

    return count

In [ ]:
PROPERTY_ID = os.getenv("GA4_PROPERTY_ID")
run_realtime_report(PROPERTY_ID)

In [ ]:
def run_realtime_report_with_minute_ranges(property_id="YOUR-GA4-PROPERTY-ID"):
    """Runs a realtime report on a Google Analytics 4 property. Dimensions
    field is omitted in the query, which results in total values of active users
    returned for each minute range in the report.

    Note the `dateRange` dimension added to the report response automatically
    as a result of querying multiple minute ranges.
    """
    client = BetaAnalyticsDataClient()

    request = RunRealtimeReportRequest(
        property=f"properties/{property_id}",
        metrics=[Metric(name="activeUsers")],
        minute_ranges=[
            MinuteRange(name="1-30 minutes ago", start_minutes_ago=30),
        ],
    )
    response = client.run_realtime_report(request)
    print_run_report_response(response)